In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    average_precision_score
)


In [2]:
csv_path = "flight_weather_merged.csv"
df = pd.read_csv(csv_path)

print("✅ CSV 로드 완료")
print("행 수:", len(df))
print("컬럼 수:", df.shape[1])
df.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_15352\1893049337.py:2: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✅ CSV 로드 완료
행 수: 1262522
컬럼 수: 52


,AIRLINE_ENGLISH,AIRLINE_KOREAN,AIRPORT,AIR_FLN,ARRIVED_ENG,ARRIVED_KOR,BAGGAGE_CLAIM,BOARDING_ENG,BOARDING_KOR,CITY,...,3시간신적설(cm),중하층운량(10분위),현상번호(국내식),지면온도(°C),풍속_ms,풍향_deg,시정_m,전운량_okta,1층 운고(FT)_결측여부,weather_datetime
0,AIR SEOUL,에어서울,GMP,RS901,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00
1,JINAIR,진에어,GMP,LJ301,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00
2,ASIANA AIRLINE,아시아나항공,GMP,OZ8937,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00
3,EASTAR JET,이스타항공,GMP,ZE201,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00
4,JINAIR,진에어,GMP,LJ303,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00


In [3]:
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "결항", "회항"]):
        return 1
    return 0

df["is_delay"] = df["RMK_KOR"].apply(make_delay_label)
df["is_delay"].value_counts()


0    1250909
1      11613
Name: is_delay, dtype: int64

In [4]:
df["flight_datetime"] = pd.to_datetime(df["flight_datetime"])

df["weekday"] = df["flight_datetime"].dt.weekday
df["month"] = df["flight_datetime"].dt.month
df["hour"] = df["flight_datetime"].dt.hour
df["flight_date"] = df["flight_datetime"].dt.date


In [5]:
num_cols = [
    "적설(cm)",
    "풍속_ms",
    "기온(°C)",
    "전운량_okta",
    "강수량(mm)",
    "STD_clean",
    "hour",
    "1층 운고(FT)",
    "이슬점온도(°C)",
    "습도(%)"
]


In [6]:
cat_cols = [
    "AIRLINE_KOREAN",
    "BOARDING_KOR",
    "ARRIVED_KOR",
    "LINE",
    "IO",
    "일기현상",
    "1층 운형"
]


In [7]:
# 실제 존재하는 컬럼만 사용
num_cols = [c for c in num_cols if c in df.columns]
cat_cols = [c for c in cat_cols if c in df.columns]

print("📊 수치형:", num_cols)
print("🏷 범주형:", cat_cols)


📊 수치형: ['적설(cm)', '풍속_ms', '기온(°C)', '전운량_okta', '강수량(mm)', 'STD_clean', 'hour', '1층 운고(FT)', '이슬점온도(°C)', '습도(%)']
🏷 범주형: ['AIRLINE_KOREAN', 'BOARDING_KOR', 'ARRIVED_KOR', 'LINE', 'IO', '일기현상', '1층 운형']


In [8]:
# 수치형 → 중앙값
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# 범주형 → UNKNOWN
for col in cat_cols:
    df[col] = df[col].fillna("UNKNOWN")


In [9]:
df = df.sort_values("flight_date")
split_date = df["flight_date"].quantile(0.8)

train_df = df[df["flight_date"] <= split_date]
test_df  = df[df["flight_date"] > split_date]

print("Train:", len(train_df), "Test:", len(test_df))


Train: 1010666 Test: 251856


In [10]:
X_train = train_df[num_cols + cat_cols]
y_train = train_df["is_delay"]

X_test  = test_df[num_cols + cat_cols]
y_test  = test_df["is_delay"]


In [11]:
class_weight = {0: 1, 1: 70}

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols)
    ]
)

model = Pipeline([
    ("prep", preprocessor),
    ("clf", LogisticRegression(
        max_iter=3000,
        class_weight=class_weight,
        n_jobs=-1
    ))
])


In [12]:
model.fit(X_train, y_train)
print("✅ 경량 모델 학습 완료")


✅ 경량 모델 학습 완료


In [13]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("PR-AUC :", average_precision_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       1.00      0.90      0.95    250885
           1       0.02      0.48      0.04       971

    accuracy                           0.90    251856
   macro avg       0.51      0.69      0.49    251856
weighted avg       0.99      0.90      0.94    251856

ROC-AUC: 0.7791604742897066
PR-AUC : 0.026662472201205198


In [14]:
for t in [0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f"\n===== Threshold = {t} =====")
    y_pred_t = (y_prob >= t).astype(int)
    print(classification_report(y_test, y_pred_t))



===== Threshold = 0.3 =====
              precision    recall  f1-score   support

           0       1.00      0.73      0.84    250885
           1       0.01      0.70      0.02       971

    accuracy                           0.73    251856
   macro avg       0.50      0.71      0.43    251856
weighted avg       0.99      0.73      0.84    251856


===== Threshold = 0.4 =====
              precision    recall  f1-score   support

           0       1.00      0.84      0.91    250885
           1       0.01      0.58      0.03       971

    accuracy                           0.83    251856
   macro avg       0.51      0.71      0.47    251856
weighted avg       0.99      0.83      0.91    251856


===== Threshold = 0.5 =====
              precision    recall  f1-score   support

           0       1.00      0.90      0.95    250885
           1       0.02      0.48      0.04       971

    accuracy                           0.90    251856
   macro avg       0.51      0.69      0.